>Conda Environment: `blockchain39` (iMac)

--- 
# Mammoth NFT
---  
> 🏁🏎  build an NFT smart contract - compile it - deploy it to the ethereum testnet - interact with it (mint NFTs)   
## Source: 
The code follows these 5 tutorials from Mammothinteractive:
- https://www.youtube.com/watch?v=SQ-chPSNgGw&list=PLw-9a9yL-pt3oKQgq_ZYzkMKAkbr8M67P&index=10
- https://www.youtube.com/watch?v=HOJ-Xl9_FYg&list=PLw-9a9yL-pt3oKQgq_ZYzkMKAkbr8M67P&index=11
- https://www.youtube.com/watch?v=lL5s9FDYo64&list=PLw-9a9yL-pt3oKQgq_ZYzkMKAkbr8M67P&index=12
- https://www.youtube.com/watch?v=lL5s9FDYo64&list=PLw-9a9yL-pt3oKQgq_ZYzkMKAkbr8M67P&index=12
- https://www.youtube.com/watch?v=DjMoJEnnvXs&list=PLw-9a9yL-pt3oKQgq_ZYzkMKAkbr8M67P&index=13

_`MammothNFT` are - if I understand it right - NFT collectibles of mammoths: the user basically creates a herd of mammoths, e.g. every new mammoth must be minted (and depends on father and mother mammoth), can be owned and ownership can be transferred.    
It is basically not so different to our `HelloWorld` example. It starts with a smart contract and shows how to deploy and interact with it. But we could use this to design our own NFTs collectibles (instead of mammoths we can create something else: puppy breeds, aliens, faultier (everybody loves 🦥), or ...)_

> The code from the videos did not work -> I substitated some parts with different code. Especially the `EthereumTesterProvider` was not working, but might be useful for us: it provides a nice testing evnironment where the students dont need a Metamask wallet but can write and deploy to the testnet.

In [29]:
# IMPORT LIBRARIES

# Standard library imports
import sys
import os
import pkg_resources
import time as time

# Third party imports
import solcx
from web3 import Web3 
from web3.auto.gethdev import w3
from web3 import EthereumTesterProvider
from Crypto.Hash import keccak
from dotenv import load_dotenv

# UnpackAI Trainings Module
import bl101

# Constants
ENVIRONMENT = pkg_resources.resource_filename('bl101', '.env')
CONTRACT_DIR = pkg_resources.resource_filename("bl101", "contracts")

# Load environment variables from .env in bl101
load_dotenv(ENVIRONMENT)

# LOAD METAMASK CREDENTIALS from local file
load_dotenv('.privatecredentials')
private_key = "0x" + os.getenv("private_key_A")


In [31]:
# Load nft contract from bl101 package (I puth the contract into our package)
contract_file = os.path.join(CONTRACT_DIR, 'MammothNFT.sol')
with open(contract_file, "r") as file:
    contract = file.read()

In [ ]:
# Compile it
solcx.install_solc('0.8.12') # install the right solidity version

compiled_contract = solcx.compile_standard(
    {
    "language": "Solidity",
    "sources": {"helloworld.sol": {"content": contract}},
    "settings": {"outputSelection": {"*": {"*": ["abi", "metadata", "evm.bytecode", "evm.bytecode.sourceMap"] }}},
    
    },
    solc_version = "0.8.12"
)
# -> creates dict with keys: errors, sources & >>>contracts<<<

In [32]:
compiled_solidity = solcx.compile_source(contract, output_values = ['abi', 'bin'])

In [33]:
# compiled_solidity contains all the information we need - we now only grab the part we really need:
contract_id, contract_interface = compiled_solidity.popitem()

In [34]:
contract_id

'<stdin>:MammothNFT'

The `contract_interface` contains the ABI. And like in our Helloworld contract from course 1 the abi contains all the metadata of the contract in json-format:

In [35]:
abi = contract_interface["abi"]
#abi

And again we need the bytecode, i.e. the (for humans) non-readable code which can be converted back to solidity and which we send then to the blockchain. We can grab it from the `contract_interface`:

In [36]:
# and again we need the bytecode of the contract:
bytecode = contract_interface['bin']

... of course the bytecode is not readable but we can print out length and the first 500 character:

In [37]:
print(len(bytecode)) #maybe we should refer to the HelloWorld contract length and show how much more the gas price now is
bytecode[:500]

8394


'608060405234801561001057600080fd5b50611045806100206000396000f3fe608060405234801561001057600080fd5b506004361061009e5760003560e01c806370a082311161006657806370a082311461016f57806391f8866e1461019f57806395d89b41146101cf578063a9059cbb146101ed578063e9716b66146102095761009e565b80630137b84f146100a357806306fdde03146100d357806318160ddd146100f15780635d6bd20a1461010f5780636352211e1461013f575b600080fd5b6100bd60048036038101906100b89190610b26565b610239565b6040516100ca9190610b94565b60405180910390f35b6100db61026c'

## Deploy to `Ethereum` blockchain

In [ ]:
# WAS NOT RUNNING
# alternative to infura: EthereumTesterProvider:
"""
w3 = Web3(Web3.EthereumTesterProvider())
w3.eth.default_account = w3.eth.accounts[0]
w3.eth.default_account
w3
"""

/Users/dirkkalmbach/opt/anaconda3/envs/blockchain39/lib/python3.9/site-packages/eth_tester/backends/__init__.py:30: UserWarning: Ethereum Tester: No backend was explicitly set, and no *full* backends were available.  Falling back to the `MockBackend` which does not support all EVM functionality.  Please refer to the `eth-tester` documentation for information on what backends are available and how to set them.  Your py-evm package may need to be updated.
  warnings.warn(UserWarning(


In [39]:
# So I used again Rinkeby Testnet via Infura
infura_url_rinkeby = 'https://rinkeby.infura.io/v3/' + os.environ.get("INFURA")  #my Infura project ID/uri for Rinkeby (stored in bl101)

w3 = Web3(Web3.HTTPProvider(infura_url_rinkeby))
w3.isConnected()

True

Last time we used an http-provder (infura). Now we use the `EthereumTesterProvider` - a useful ...

In [40]:
# build contract from abi and bytecode
NFT = w3.eth.contract(abi=abi, bytecode=bytecode)
NFT

web3._utils.datatypes.Contract

In [41]:
# Get Nonce from wallet
wallet_address = os.environ.get("WALLETADDRESS") # <-from bl101, my personal Metamask account (? shall we provide one unpackai metamask account for the students?)
nonce = w3.eth.get_transaction_count(wallet_address)
nonce

137

Perform a transaction.

In [42]:
rinkeby_chain = 4
transaction = NFT.constructor().buildTransaction(
    {
        "chainId": rinkeby_chain, 
        "from": wallet_address, 
        "nonce": nonce, 
        "gasPrice": w3.eth.gas_price})

In [43]:
signed_transaction = w3.eth.account.sign_transaction(transaction, private_key=private_key)

In [44]:
transaction_hash = w3.eth.send_raw_transaction(signed_transaction.rawTransaction)

In [45]:
transaction_receipt = w3.eth.wait_for_transaction_receipt(transaction_hash)

In [46]:
nft_object = w3.eth.contract(address=transaction_receipt.contractAddress, abi=abi)
nft_object

## Interact with smart contract [📺](https://youtu.be/DjMoJEnnvXs?list=PLw-9a9yL-pt3oKQgq_ZYzkMKAkbr8M67P)

`nft_object` is a reference to our nft object on the blockchain. We can call any function inside this contract as long as they are public or external. The syntaxt for acessing functions are:   
>*[smart contract object]*.`functions`.*[function name]*`(`*[parameters (optional)]*`)`.`call()`

Call are acessing a value or trying out a transaction, but its not a official transaction (no gas needed)

In [47]:
# reference to nft object which exists on the blockchain = nft_object
nft_object.functions.totalSupply().call() #calls the totalSupply function the smart contract

0

We can increase that supply (I think with supply they mean the size of the herd) we need to mint an NFT:

`_createMammoth` is the public function to mint the NFT. It has these parameters:
- _motherID
- _father_ID
- _generation
- _genes
_ _owner
and returns `newMammothId`, which is an address and will become the owner of the NFT. 


*(maybe it would make sense to look into the contract at this time to understand also the other functions)*

In [49]:
# we mint it to ourself
# this is not making a transaction (just a call) -> does not cost gas
nft_object.functions._createMammoth(1, 2, 3, 12342, wallet_address).call()

0

### Minting a 🐘

Make a transaction:

In [50]:
# nonce from my wallet 
nonce = w3.eth.get_transaction_count(wallet_address)

# instead of call function, we build a transaction function:
update_nft_transaction = nft_object.functions._createMammoth(1, 2, 3, 12342, wallet_address).buildTransaction(
    {
        "gasPrice": w3.eth.gas_price,
        "chainId": rinkeby_chain,
        "from": wallet_address,
        "nonce": nonce
    }
)


In [ ]:
# from: my wallet to: contract address
update_nft_transaction

{'value': 0,
 'gas': 165133,
 'gasPrice': 1025981699,
 'chainId': 4,
 'from': '0x7Dda3ae0b40D7d86AC54edc7888c40EF7521c3c0',
 'nonce': 136,
 'to': '0xcB1AEd0F4BeB0B5749E096D109e37eF4Da6D903e',
 'data': '0x91f8866e00000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000300000000000000000000000000000000000000000000000000000000000030360000000000000000000000007dda3ae0b40d7d86ac54edc7888c40ef7521c3c0'}

![contract on ethereum](images/mint-mammoth-contract.png)

In [51]:
# this might take a while (depending on how big the contract is)
t0=time.time()

# 1.
signed_transaction = w3.eth.account.sign_transaction(update_nft_transaction,
                                                        private_key=private_key)
# 2.
transaction_hash = w3.eth.send_raw_transaction(signed_transaction.rawTransaction)

# 3.
transaction_receipt = w3.eth.wait_for_transaction_receipt(transaction_hash)

t1=time.time()
print(f"Execution time: {t1-t0:.2f}s")

Execution time: 16.91s


In [53]:
# If we want we can printout the details of the transaction
transaction_receipt

AttributeDict({'blockHash': HexBytes('0x7a096e43dcdc07f606b8ee6e8fecc82b90e2f79cadea2bb1731509738a16f195'),
 'blockNumber': 10668583,
 'contractAddress': None,
 'cumulativeGasUsed': 4680653,
 'effectiveGasPrice': 1025605415,
 'from': '0x7Dda3ae0b40D7d86AC54edc7888c40EF7521c3c0',
 'gasUsed': 165133,
 'logs': [AttributeDict({'address': '0xFb0D532e28233f989986bBD5fA2D89b43F98AE49',
   'blockHash': HexBytes('0x7a096e43dcdc07f606b8ee6e8fecc82b90e2f79cadea2bb1731509738a16f195'),
   'blockNumber': 10668583,
   'data': '0x0000000000000000000000007dda3ae0b40d7d86ac54edc7888c40ef7521c3c00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000003036',
   'logIndex': 56,
   'removed': False,
   'topics': [HexBytes('0x0a5311bd2a6608f08a180df2ee7c5946819a649b204b554bb8e39825b2c50ad5')],
   'transactionHash': 

Rerun again `totalSuplly` function of our contract:

In [55]:
nft_object.functions.totalSupply().call()

1

È voilà! `totalSupply` increased by 1! 🥳🎊🎉